In [20]:
# !pip install joblib

In [21]:
# !pip install node2vec

In [5]:
import os
import json
import pandas as pd
import numpy as np
import networkx as nx
import joblib
import warnings
from node2vec import Node2Vec
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. 아티팩트 및 환경 설정
# ---------------------------------------------------------
DATA_DIR = r'./JSON 모음'
model_nlp = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

LEAGUE_MAP = {
    "K리그": "K league",
    "EPL": "EPL",
    "KBO": "KBO",
    "F1": "F1"
}

try:
    artifacts = joblib.load('sports_chatbot_model2.joblib')
    final_model = artifacts['final_model']
    pca = artifacts['pca']
    scaler = artifacts['scaler']
    le_team = artifacts['le_team']
    le_league = artifacts['le_league']
    input_features = artifacts['input_features']
    print("✅ 모델 및 전처리 아티팩트 로드 완료")
except Exception as e:
    print(f"❌ 로드 실패: {e}")
    exit()

# ---------------------------------------------------------
# 2. 데이터 로딩 및 N2V 구축
# ---------------------------------------------------------
def load_teams(path):
    teams = []
    if not os.path.exists(path): return teams
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith('.json'):
                with open(os.path.join(root, filename), 'r', encoding='utf-8') as f:
                    teams.append(json.load(f))
    return teams

def build_n2v_model(teams_data):
    G = nx.Graph()
    for t in teams_data:
        if 'team_name' in t: G.add_node(t['team_name'])
    
    for i in range(len(teams_data)):
        for j in range(i + 1, len(teams_data)):
            tags1 = set(teams_data[i].get('style_tags', []))
            tags2 = set(teams_data[j].get('style_tags', []))
            common = len(tags1.intersection(tags2))
            if common > 0:
                G.add_edge(teams_data[i]['team_name'], teams_data[j]['team_name'], weight=common)
    
    n2v = Node2Vec(G, dimensions=64, walk_length=10, num_walks=40, workers=1, quiet=True)
    return n2v.fit(window=5, min_count=1)

print("🚀 데이터를 분석 중입니다...")
teams_master = load_teams(DATA_DIR)
n2v_model = build_n2v_model(teams_master)

# ---------------------------------------------------------
# 3. 실시간 점수 계산 (N2V 유사도 반영)
# ---------------------------------------------------------
def get_scores_strict(query, anchor_name, candidate):
    # sbert_score
    cand_tags = " ".join(candidate.get('style_tags', []))
    embs = model_nlp.encode([query, cand_tags])
    s_sem = cosine_similarity([embs[0]], [embs[1]])[0][0]

    # n2v_score: 응원 팀이 있을 때만 모델에서 관계 점수 산출
    s_rel = 0.5
    if anchor_name and anchor_name in n2v_model.wv and candidate['team_name'] in n2v_model.wv:
        s_rel = n2v_model.wv.similarity(anchor_name, candidate['team_name'])

    # vector_score (7대 지표 기반)
    ts = candidate.get('scores', {})
    metrics = ['strength', 'money', 'star_power', 'attack_style', 'underdog_feel', 'fan_passion', 'tradition']
    t_vec = np.array([ts.get(m, 10) for m in metrics])
    s_vec = cosine_similarity(np.array([10]*7).reshape(1, -1), t_vec.reshape(1, -1))[0][0]
    
    return s_sem, s_rel, s_vec

# ---------------------------------------------------------
# 4. 추천 서비스 (메인 로직)
# ---------------------------------------------------------
def recommend_service(query, user_type, support_team, target_league):
    json_league_name = LEAGUE_MAP.get(target_league, target_league)
    
    candidates = [t for t in teams_master if t.get('league', '').lower() == json_league_name.lower()]
    if not candidates: return f"'{target_league}'에 해당하는 팀을 찾을 수 없습니다."

    query_pca = pca.transform(model_nlp.encode([query]))[0]
    pca_cols = ["의도_팬덤정체성", "의도_스타성과강함", "의도_명문과기적", "의도_비주얼과매력", "의도_자본과지배력"]

    rows = []
    for cand in candidates:
        s_sem, s_rel, s_vec = get_scores_strict(query, support_team, cand)
        row = {
            'matching_team': cand['team_name'],
            'user_type': int(user_type),
            'recommend_league': json_league_name,
            'sbert_score': s_sem,
            'n2v_score': s_rel,
            'vector_score': s_vec
        }
        for i, col in enumerate(pca_cols): row[col] = query_pca[i]
        rows.append(row)

    df_inf = pd.DataFrame(rows)
    df_inf['recommend_league_enc'] = le_league.transform(df_inf['recommend_league'].astype(str))
    
    score_cols = ['sbert_score', 'n2v_score', 'vector_score']
    scaled = scaler.transform(df_inf[score_cols])
    df_inf['sbert_score_mm'], df_inf['n2v_score_mm'], df_inf['vector_score_mm'] = scaled[:,0], scaled[:,1], scaled[:,2]

    X_input = df_inf[input_features]
    cat_cols = [c for c in ["user_type", "recommend_league_enc"] if c in input_features]
    for c in cat_cols:
        X_input[c] = X_input[c].astype('category')

    df_inf['predict_score'] = final_model.predict(X_input)
    return df_inf.sort_values(by='predict_score', ascending=False)

# ---------------------------------------------------------
# 5. 실행 및 입력값 출력
# ---------------------------------------------------------


print("\n" + "="*50)
user_q = input("💬 어떤 스타일의 팀을 선호하시나요? (예: 돈은 없지만 열정적인 팀): ")
ut_raw = input("💬 스포츠 팬이신가요? (1:팬, 0:초보): ")

my_team = None
if ut_raw == '1':
    my_team = input("💬 현재 응원하고 계신 팀 이름을 입력해주세요: ")

target_raw = input("💬 어떤 리그의 팀을 추천받고 싶으신가요? (EPL/K리그/KBO/F1): ")

# 추천 실행
res = recommend_service(user_q, ut_raw, my_team, target_raw)

# 결과 대시보드 출력
print("\n" + "▣"*25)
print("       [입력값 및 추천 분석 리포트]       ")
print("▣"*25)
print(f"▶ 선호 스타일    : {user_q}")
print(f"▶ 사용자 유형    : {'기존 팬 (Fan)' if ut_raw == '1' else '입문자 (Beginner)'}")
if ut_raw == '1':
    print(f"▶ 기존 응원팀    : {my_team}")
print(f"▶ 추천 대상 리그 : {target_raw}")
print("-" * 50)

if isinstance(res, str):
    print(f"⚠️ 결과: {res}")
else:
    print(f"✨ 추천 결과 (상위 5개 팀):")
    # itertuples()를 통해 결과 행들을 순회합니다.
    for i, row in enumerate(res.head(5).itertuples(), 1):
        print(f"{i}위: [{row.matching_team}]")
        print(f"   - 정합도 점수: {row.predict_score:.4f} (높을수록 좋음)")
        # 응원팀이 있을 때만 n2v_score의 의미가 큽니다.
        if ut_raw == '1':
            print(f"   - 관계 유사도(n2v): {row.n2v_score:.4f}")
        print("-" * 35)

print("▣"*25)

✅ 모델 및 전처리 아티팩트 로드 완료
🚀 데이터를 분석 중입니다...


▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣
       [입력값 및 추천 분석 리포트]       
▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣
▶ 선호 스타일    : 오렌지하우스
▶ 사용자 유형    : 입문자 (Beginner)
▶ 추천 대상 리그 : F1
--------------------------------------------------
✨ 추천 결과 (상위 5개 팀):
1위: [레드불]
   - 정합도 점수: -0.8768 (높을수록 좋음)
-----------------------------------
2위: [하스]
   - 정합도 점수: -2.5507 (높을수록 좋음)
-----------------------------------
3위: [맥라렌]
   - 정합도 점수: -2.8712 (높을수록 좋음)
-----------------------------------
4위: [윌리엄스]
   - 정합도 점수: -3.3908 (높을수록 좋음)
-----------------------------------
5위: [자우버]
   - 정합도 점수: -3.6006 (높을수록 좋음)
-----------------------------------
▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣▣


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import networkx as nx
import joblib
import warnings
from node2vec import Node2Vec
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. 아티팩트 및 환경 설정
# ---------------------------------------------------------
DATA_DIR = r'./JSON 모음'
model_nlp = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# [추가] score.py의 로직 가중치 설정
ALPHA, BETA, GAMMA = 0.4, 0.4, 0.2

LEAGUE_MAP = {
    "K리그": "K league",
    "EPL": "EPL",
    "KBO": "KBO",
    "F1": "F1"
}

try:
    artifacts = joblib.load('sports_chatbot_model2.joblib')
    final_model = artifacts['final_model']
    pca = artifacts['pca']
    scaler = artifacts['scaler']
    le_team = artifacts['le_team']
    le_league = artifacts['le_league']
    input_features = artifacts['input_features']
    print("✅ 모델 및 전처리 아티팩트 로드 완료")
except Exception as e:
    print(f"❌ 로드 실패: {e}")
    exit()

# ---------------------------------------------------------
# 2. 데이터 로딩 및 N2V 구축 (기존 유지)
# ---------------------------------------------------------
def load_teams(path):
    teams = []
    if not os.path.exists(path): return teams
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith('.json'):
                with open(os.path.join(root, filename), 'r', encoding='utf-8') as f:
                    teams.append(json.load(f))
    return teams

def build_n2v_model(teams_data):
    G = nx.Graph()
    for t in teams_data:
        if 'team_name' in t: G.add_node(t['team_name'])
    
    for i in range(len(teams_data)):
        for j in range(i + 1, len(teams_data)):
            tags1 = set(teams_data[i].get('style_tags', []))
            tags2 = set(teams_data[j].get('style_tags', []))
            common = len(tags1.intersection(tags2))
            if common > 0:
                G.add_edge(teams_data[i]['team_name'], teams_data[j]['team_name'], weight=common)
    
    n2v = Node2Vec(G, dimensions=64, walk_length=10, num_walks=40, workers=1, quiet=True)
    return n2v.fit(window=5, min_count=1)

print("🚀 데이터를 분석 중입니다...")
teams_master = load_teams(DATA_DIR)
n2v_model = build_n2v_model(teams_master)

# ---------------------------------------------------------
# 3. 실시간 점수 계산 (anchor_name 반영)
# ---------------------------------------------------------
def get_scores_strict(query, anchor_name, candidate):
    # sbert_score
    cand_tags = " ".join(candidate.get('style_tags', []))
    embs = model_nlp.encode([query, cand_tags])
    s_sem = cosine_similarity([embs[0]], [embs[1]])[0][0]

    # n2v_score
    s_rel = 0.5
    if anchor_name and anchor_name in n2v_model.wv and candidate['team_name'] in n2v_model.wv:
        s_rel = n2v_model.wv.similarity(anchor_name, candidate['team_name'])

    # vector_score
    ts = candidate.get('scores', {})
    metrics = ['strength', 'money', 'star_power', 'attack_style', 'underdog_feel', 'fan_passion', 'tradition']
    t_vec = np.array([ts.get(m, 10) for m in metrics])
    # 기준 벡터와의 코사인 유사도 (모두 10점인 이상적인 팀 기준)
    s_vec = cosine_similarity(np.array([10]*7).reshape(1, -1), t_vec.reshape(1, -1))[0][0]
    
    return s_sem, s_rel, s_vec

# ---------------------------------------------------------
# 4. 추천 서비스 (5:5 하이브리드 로직 적용)
# ---------------------------------------------------------
def recommend_service(query, user_type, support_team, target_league):
    json_league_name = LEAGUE_MAP.get(target_league, target_league)
    
    candidates = [t for t in teams_master if t.get('league', '').lower() == json_league_name.lower()]
    if not candidates: return f"'{target_league}'에 해당하는 팀을 찾을 수 없습니다."

    query_pca = pca.transform(model_nlp.encode([query]))[0]
    pca_cols = ["의도_팬덤정체성", "의도_스타성과강함", "의도_명문과기적", "의도_비주얼과매력", "의도_자본과지배력"]

    rows = []
    for cand in candidates:
        s_sem, s_rel, s_vec = get_scores_strict(query, support_team, cand)
        
        # [핵심] score.py 방식의 수동 매칭 점수 계산
        manual_match_score = (s_sem * ALPHA) + (s_rel * BETA) + (s_vec * GAMMA)

        row = {
            'matching_team': cand['team_name'],
            'user_type': int(user_type),
            'recommend_league': json_league_name,
            'sbert_score': s_sem,
            'n2v_score': s_rel,
            'vector_score': s_vec,
            'manual_match_score': manual_match_score  # 수동 점수 저장
        }
        for i, col in enumerate(pca_cols): row[col] = query_pca[i]
        rows.append(row)

    df_inf = pd.DataFrame(rows)
    df_inf['recommend_league_enc'] = le_league.transform(df_inf['recommend_league'].astype(str))
    
    # 모델 입력용 스케일링
    score_cols = ['sbert_score', 'n2v_score', 'vector_score']
    scaled = scaler.transform(df_inf[score_cols])
    df_inf['sbert_score_mm'], df_inf['n2v_score_mm'], df_inf['vector_score_mm'] = scaled[:,0], scaled[:,1], scaled[:,2]

    # 모델 예측
    X_input = df_inf[input_features]
    cat_cols = [c for c in ["user_type", "recommend_league_enc"] if c in input_features]
    for c in cat_cols:
        X_input[c] = X_input[c].astype('category')

    df_inf['predict_score'] = final_model.predict(X_input)

    # [핵심] 5:5 최종 점수 합산 (Hybrid Score)
    # 두 점수의 스케일이 다를 수 있으므로 정규화를 고려할 수 있으나, 기본적으로 5:5 결합 수행
    df_inf['final_hybrid_score'] = (df_inf['manual_match_score'] * 0.8) + (df_inf['predict_score'] * 0.2)

    return df_inf.sort_values(by='final_hybrid_score', ascending=False)

# ---------------------------------------------------------
# 5. 실행부 (결과 출력 수정)
# ---------------------------------------------------------
print("\n" + "="*50)
user_q = input("💬 어떤 스타일의 팀을 선호하시나요?: ")
ut = input("💬 스포츠 팬이신가요? (1:팬, 0:초보): ")

my_team = None
if ut == '1':
    print(f"현재 등록된 팀 예시: {', '.join([t['team_name'] for t in teams_master[:5]])} 등")
    my_team = input("💬 현재 응원하고 계신 팀 이름을 입력해주세요: ")

target = input("💬 어떤 리그의 팀을 추천받고 싶으신가요? (EPL/K리그/KBO/F1): ")

res = recommend_service(user_q, ut, my_team, target)

if isinstance(res, str):
    print(f"\n⚠️ {res}")
else:
    print(f"\n✨ 당신을 위한 '{target}' 리그 5:5 하이브리드 추천 순위입니다:")
    for i, row in enumerate(res.itertuples(), 1):
        star = "⭐" if i <= 3 else "  "
        print(f"{star} {i}위: {row.matching_team}")
        print(f"    [최종 점수: {row.final_hybrid_score:.4f}] (수동: {row.manual_match_score:.4f} / 모델: {row.predict_score:.4f})")

✅ 모델 및 전처리 아티팩트 로드 완료
🚀 데이터를 분석 중입니다...


✨ 당신을 위한 'K LEAGUE' 리그 5:5 하이브리드 추천 순위입니다:
⭐ 1위: FC 서울
    [최종 점수: 0.6819] (수동: 0.5396 / 모델: 1.2509)
⭐ 2위: 전북 현대 모터스
    [최종 점수: 0.5291] (수동: 0.5697 / 모델: 0.3670)
⭐ 3위: 인천 유나이티드
    [최종 점수: 0.4166] (수동: 0.5724 / 모델: -0.2064)
   4위: 광주FC
    [최종 점수: 0.3756] (수동: 0.5449 / 모델: -0.3016)
   5위: 대전 하나 시티즌
    [최종 점수: 0.3589] (수동: 0.5574 / 모델: -0.4350)
   6위: 수원 삼성 블루윙즈
    [최종 점수: 0.2261] (수동: 0.5739 / 모델: -1.1652)
   7위: 대구FC
    [최종 점수: 0.2087] (수동: 0.5389 / 모델: -1.1123)
   8위: 강원FC
    [최종 점수: 0.1913] (수동: 0.5655 / 모델: -1.3054)
   9위: 제주 SK FC
    [최종 점수: 0.1688] (수동: 0.5238 / 모델: -1.2513)
   10위: 울산 HD FC
    [최종 점수: 0.1256] (수동: 0.5370 / 모델: -1.5197)
   11위: 포항 스틸러스
    [최종 점수: 0.1020] (수동: 0.5595 / 모델: -1.7282)
   12위: FC안양
    [최종 점수: -0.0366] (수동: 0.5635 / 모델: -2.4370)


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import networkx as nx
import joblib
import warnings
from node2vec import Node2Vec
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. 아티팩트 및 환경 설정
# ---------------------------------------------------------
DATA_DIR = r'./JSON 모음'
model_nlp = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# score.py의 로직 가중치 설정
ALPHA, BETA, GAMMA = 0.4, 0.4, 0.2

LEAGUE_MAP = {
    "K리그": "K league",
    "EPL": "EPL",
    "KBO": "KBO",
    "F1": "F1"
}

try:
    artifacts = joblib.load('sports_chatbot_model2.joblib')
    final_model = artifacts['final_model']
    pca = artifacts['pca']
    scaler = artifacts['scaler']
    le_team = artifacts['le_team']
    le_league = artifacts['le_league']
    input_features = artifacts['input_features']
    print("✅ 모델 및 전처리 아티팩트 로드 완료")
except Exception as e:
    print(f"❌ 로드 실패: {e}")
    exit()

# ---------------------------------------------------------
# 2. 데이터 로딩 및 N2V 구축
# ---------------------------------------------------------
def load_teams(path):
    teams = []
    if not os.path.exists(path): return teams
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith('.json'):
                with open(os.path.join(root, filename), 'r', encoding='utf-8') as f:
                    teams.append(json.load(f))
    return teams

def build_n2v_model(teams_data):
    G = nx.Graph()
    for t in teams_data:
        if 'team_name' in t: G.add_node(t['team_name'])
    
    for i in range(len(teams_data)):
        for j in range(i + 1, len(teams_data)):
            tags1 = set(teams_data[i].get('style_tags', []))
            tags2 = set(teams_data[j].get('style_tags', []))
            common = len(tags1.intersection(tags2))
            if common > 0:
                G.add_edge(teams_data[i]['team_name'], teams_data[j]['team_name'], weight=common)
    
    n2v = Node2Vec(G, dimensions=64, walk_length=10, num_walks=40, workers=1, quiet=True)
    return n2v.fit(window=5, min_count=1)

print("🚀 데이터를 분석 중입니다...")
teams_master = load_teams(DATA_DIR)
n2v_model = build_n2v_model(teams_master)

# ---------------------------------------------------------
# 3. 실시간 점수 계산 (시나리오 로직 적용 버전)
# ---------------------------------------------------------
def get_scores_strict(query, anchor_name, candidate):
    # (1) sbert_score
    cand_tags = " ".join(candidate.get('style_tags', []))
    embs = model_nlp.encode([query, cand_tags])
    s_sem = cosine_similarity([embs[0]], [embs[1]])[0][0]

    # (2) n2v_score
    s_rel = 0.5
    if anchor_name and anchor_name in n2v_model.wv and candidate['team_name'] in n2v_model.wv:
        s_rel = n2v_model.wv.similarity(anchor_name, candidate['team_name'])

    # (3) vector_score [리더님의 시나리오 로직 100% 적용]
    ts = candidate.get('scores', {})
    metrics = ['strength', 'money', 'star_power', 'attack_style', 'underdog_feel', 'fan_passion', 'tradition']
    t_vec = np.array([ts.get(m, 10) for m in metrics])
    
    target_vec = np.array([10]*7)
    s_multiplier = 1.0

    # --- 요청하신 시나리오 조건문 시작 ---
    if any(k in query for k in ["언더독", "기적", "저비용", "머니볼", "효율", "가성비"]):
        target_vec[4] = 50
        if ts.get('money', 10) >= 16: s_multiplier = 0.3
        elif ts.get('money', 10) <= 8: s_multiplier = 1.5
    elif any(k in query for k in ["강한", "압도적", "최강", "우승", "부자"]):
        target_vec[0], target_vec[1] = 40, 40
        if ts.get('strength', 10) < 12: s_multiplier = 0.4
    elif any(k in query for k in ["미남", "잘생긴", "비주얼", "얼굴", "입덕"]):
        target_vec[2] = 50
        if ts.get('star_power', 10) < 12: s_multiplier = 0.4
    elif any(k in query for k in ["전통", "명문", "역사", "연고지", "자부심"]):
        target_vec[6], target_vec[5] = 40, 40
        if ts.get('tradition', 10) < 10: s_multiplier = 0.5
    elif any(k in query for k in ["공격", "화끈", "득점", "홈런", "추월", "시원시원"]):
        target_vec[3] = 50
        if ts.get('attack_style', 10) < 12: s_multiplier = 0.5
    elif any(k in query for k in ["수비", "단단한", "실리", "역습", "질식"]):
        target_vec[4], target_vec[0] = 30, 30
        if ts.get('attack_style', 10) > 15: s_multiplier = 0.5
    # --- 요청하신 시나리오 조건문 끝 ---
    
    # 최종 벡터 점수 계산
    s_vec = cosine_similarity(target_vec.reshape(1, -1), t_vec.reshape(1, -1))[0][0]
    s_vec = s_vec * s_multiplier
    
    return s_sem, s_rel, s_vec

# ---------------------------------------------------------
# 4. 추천 서비스 (하이브리드 로직)
# ---------------------------------------------------------
def recommend_service(query, user_type, support_team, target_league):
    json_league_name = LEAGUE_MAP.get(target_league, target_league)
    
    candidates = [t for t in teams_master if t.get('league', '').lower() == json_league_name.lower()]
    if not candidates: return f"'{target_league}'에 해당하는 팀을 찾을 수 없습니다."

    query_pca = pca.transform(model_nlp.encode([query]))[0]
    pca_cols = ["의도_팬덤정체성", "의도_스타성과강함", "의도_명문과기적", "의도_비주얼과매력", "의도_자본과지배력"]

    rows = []
    for cand in candidates:
        s_sem, s_rel, s_vec = get_scores_strict(query, support_team, cand)
        
        # 수동 매칭 점수 계산 (score.py 방식)
        manual_match_score = (s_sem * ALPHA) + (s_rel * BETA) + (s_vec * GAMMA)

        row = {
            'matching_team': cand['team_name'],
            'user_type': int(user_type),
            'recommend_league': json_league_name,
            'sbert_score': s_sem,
            'n2v_score': s_rel,
            'vector_score': s_vec,
            'manual_match_score': manual_match_score
        }
        for i, col in enumerate(pca_cols): row[col] = query_pca[i]
        rows.append(row)

    df_inf = pd.DataFrame(rows)
    df_inf['recommend_league_enc'] = le_league.transform(df_inf['recommend_league'].astype(str))
    
    # 스케일링 및 모델 예측
    score_cols = ['sbert_score', 'n2v_score', 'vector_score']
    scaled = scaler.transform(df_inf[score_cols])
    df_inf['sbert_score_mm'], df_inf['n2v_score_mm'], df_inf['vector_score_mm'] = scaled[:,0], scaled[:,1], scaled[:,2]

    X_input = df_inf[input_features]
    cat_cols = [c for c in ["user_type", "recommend_league_enc"] if c in input_features]
    for c in cat_cols: X_input[c] = X_input[c].astype('category')

    df_inf['predict_score'] = final_model.predict(X_input)

    # 최종 8:2 하이브리드 결합
    df_inf['final_hybrid_score'] = (df_inf['manual_match_score'] * 0.85) + (df_inf['predict_score'] * 0.15)

    return df_inf.sort_values(by='final_hybrid_score', ascending=False)

# ---------------------------------------------------------
# 5. 실행부
# ---------------------------------------------------------
print("\n" + "="*50)
user_q = input("💬 어떤 스타일의 팀을 선호하시나요?: ")
ut = input("💬 스포츠 팬이신가요? (1:팬, 0:초보): ")

my_team = None
if ut == '1':
    print(f"현재 등록된 팀 예시: {', '.join([t['team_name'] for t in teams_master[:5]])} 등")
    my_team = input("💬 현재 응원하고 계신 팀 이름을 입력해주세요: ")

target = input("💬 어떤 리그의 팀을 추천받고 싶으신가요? (EPL/K리그/KBO/F1): ")

res = recommend_service(user_q, ut, my_team, target)

if isinstance(res, str):
    print(f"\n⚠️ {res}")
else:
    print(f"\n✨ 당신을 위한 '{target}' 리그 하이브리드 추천 순위입니다:")
    for i, row in enumerate(res.itertuples(), 1):
        star = "⭐" if i <= 3 else "  "
        print(f"{star} {i}위: {row.matching_team}")
        print(f"    [최종 점수: {row.final_hybrid_score:.4f}] (수동: {row.manual_match_score:.4f} / 모델: {row.predict_score:.4f})")

✅ 모델 및 전처리 아티팩트 로드 완료
🚀 데이터를 분석 중입니다...


✨ 당신을 위한 'K LEAGUE' 리그 하이브리드 추천 순위입니다:
⭐ 1위: FC안양
    [최종 점수: 0.5995] (수동: 0.6010 / 모델: -0.9348)
⭐ 2위: 대전 하나 시티즌
    [최종 점수: 0.5612] (수동: 0.5627 / 모델: -0.8822)
⭐ 3위: 제주 SK FC
    [최종 점수: 0.5312] (수동: 0.5335 / 모델: -1.7629)
   4위: 울산 HD FC
    [최종 점수: 0.5288] (수동: 0.5308 / 모델: -1.3829)
   5위: 광주FC
    [최종 점수: 0.5199] (수동: 0.5210 / 모델: -0.5605)
   6위: FC 서울
    [최종 점수: 0.5002] (수동: 0.5007 / 모델: 0.0304)
   7위: 전북 현대 모터스
    [최종 점수: 0.5002] (수동: 0.5019 / 모델: -1.2189)
   8위: 강원FC
    [최종 점수: 0.4953] (수동: 0.4975 / 모델: -1.6154)
   9위: 인천 유나이티드
    [최종 점수: 0.4775] (수동: 0.4808 / 모델: -2.7676)
   10위: 수원 삼성 블루윙즈
    [최종 점수: 0.4758] (수동: 0.4788 / 모델: -2.5899)
   11위: 포항 스틸러스
    [최종 점수: 0.4748] (수동: 0.4778 / 모델: -2.5899)
   12위: 대구FC
    [최종 점수: 0.4724] (수동: 0.4754 / 모델: -2.5072)


In [34]:
import os
import json
import pandas as pd
import numpy as np
import networkx as nx
import joblib
import warnings
from node2vec import Node2Vec
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. 모델 및 환경 설정 (이원화 로딩)
# ---------------------------------------------------------
DATA_DIR = r'./JSON 모음'

# [모델 1] s_sem(의미 유사도) 전용: 기존 SBERT
print("🔍 SBERT 모델(KR-SBERT) 로딩 중...")
model_sbert = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# [모델 2] PCA(의도 분석) 전용: 문맥 이해 특화 모델
print("🔍 PCA 인코더 모델(ko-sroberta) 로딩 중...")
model_pca_encoder = SentenceTransformer('jhgan/ko-sroberta-multitask')

ALPHA, BETA, GAMMA = 0.4, 0.4, 0.2

LEAGUE_MAP = {
    "K리그": "K league", "EPL": "EPL", "KBO": "KBO", "F1": "F1"
}

try:
    artifacts = joblib.load('sports_chatbot_model2.joblib')
    final_model = artifacts['final_model']
    pca = artifacts['pca']
    scaler = artifacts['scaler']
    le_league = artifacts['le_league']
    input_features = artifacts['input_features']
    print("✅ 모델 및 전처리 아티팩트 로드 완료")
except Exception as e:
    print(f"❌ 로드 실패: {e}")
    exit()

# ---------------------------------------------------------
# 2. 데이터 로딩 및 N2V 구축 (기존 유지)
# ---------------------------------------------------------
def load_teams(path):
    teams = []
    if not os.path.exists(path): return teams
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith('.json'):
                with open(os.path.join(root, filename), 'r', encoding='utf-8') as f:
                    teams.append(json.load(f))
    return teams

def build_n2v_model(teams_data):
    G = nx.Graph()
    for t in teams_data:
        if 'team_name' in t: G.add_node(t['team_name'])
    for i in range(len(teams_data)):
        for j in range(i + 1, len(teams_data)):
            tags1 = set(teams_data[i].get('style_tags', []))
            tags2 = set(teams_data[j].get('style_tags', []))
            common = len(tags1.intersection(tags2))
            if common > 0:
                G.add_edge(teams_data[i]['team_name'], teams_data[j]['team_name'], weight=common)
    n2v = Node2Vec(G, dimensions=64, walk_length=10, num_walks=40, workers=1, quiet=True)
    return n2v.fit(window=5, min_count=1)

print("🚀 팀 네트워크 분석 중...")
teams_master = load_teams(DATA_DIR)
n2v_model = build_n2v_model(teams_master)

# ---------------------------------------------------------
# 3. 실시간 점수 계산 (SBERT 유지 + 리더님 시나리오 로직)
# ---------------------------------------------------------
def get_scores_strict(user_query, anchor_name, candidate):
    # (1) sbert_score: 기존 SBERT 모델 사용
    cand_tags = " ".join(candidate.get('style_tags', []))
    embs = model_sbert.encode([user_query, cand_tags])
    s_sem = cosine_similarity([embs[0]], [embs[1]])[0][0]

    # (2) n2v_score: 관계 유사도
    s_rel = 0.5
    if anchor_name and anchor_name in n2v_model.wv and candidate['team_name'] in n2v_model.wv:
        s_rel = n2v_model.wv.similarity(anchor_name, candidate['team_name'])

    # (3) vector_score: 리더님 시나리오 로직 적용
    ts = candidate.get('scores', {})
    metrics = ['strength', 'money', 'star_power', 'attack_style', 'underdog_feel', 'fan_passion', 'tradition']
    t_vec = np.array([ts.get(m, 10) for m in metrics])
    
    target_vec = np.array([10]*7)
    s_multiplier = 1.0

    # --- 리더님 시나리오 로직 (수정 절대 없음) ---
    if any(k in user_query for k in ["언더독", "기적", "저비용", "머니볼", "효율", "가성비"]):
        target_vec[4] = 50
        if ts.get('money', 10) >= 16: s_multiplier = 0.3
        elif ts.get('money', 10) <= 8: s_multiplier = 1.5
    elif any(k in user_query for k in ["강한", "압도적", "최강", "우승", "부자"]):
        target_vec[0], target_vec[1] = 40, 40
        if ts.get('strength', 10) < 12: s_multiplier = 0.4
    elif any(k in user_query for k in ["미남", "잘생긴", "비주얼", "얼굴", "입덕", "스타"]):
        target_vec[2] = 50
        if ts.get('star_power', 10) < 12: s_multiplier = 0.4
    elif any(k in user_query for k in ["전통", "명문", "역사", "연고지", "자부심"]):
        target_vec[6], target_vec[5] = 40, 40
        if ts.get('tradition', 10) < 10: s_multiplier = 0.5
    elif any(k in user_query for k in ["공격", "화끈", "득점", "홈런", "추월", "시원시원"]):
        target_vec[3] = 50
        if ts.get('attack_style', 10) < 12: s_multiplier = 0.5
    elif any(k in user_query for k in ["수비", "단단한", "실리", "역습", "질식"]):
        target_vec[4], target_vec[0] = 30, 30
        if ts.get('attack_style', 10) > 15: s_multiplier = 0.5
    # ----------------------------------------

    s_vec = cosine_similarity(target_vec.reshape(1, -1), t_vec.reshape(1, -1))[0][0]
    s_vec = s_vec * s_multiplier
    
    return s_sem, s_rel, s_vec

# ---------------------------------------------------------
# 4. 추천 서비스 (PCA 전용 모델 사용)
# ---------------------------------------------------------
def recommend_service(query, user_type, support_team, target_league):
    json_league_name = LEAGUE_MAP.get(target_league, target_league)
    candidates = [t for t in teams_master if t.get('league', '').lower() == json_league_name.lower()]
    
    if not candidates: return f"'{target_league}' 데이터 없음"

    # [수정] PCA용 벡터화만 ko-sroberta 모델 사용
    query_pca_vec = model_pca_encoder.encode([query])
    query_pca = pca.transform(query_pca_vec)[0]
    pca_cols = ["의도_팬덤정체성", "의도_스타성과강함", "의도_명문과기적", "의도_비주얼과매력", "의도_자본과지배력"]

    rows = []
    for cand in candidates:
        s_sem, s_rel, s_vec = get_scores_strict(query, support_team, cand)
        
        # 수동 점수 합산
        manual_match_score = (s_sem * ALPHA) + (s_rel * BETA) + (s_vec * GAMMA)

        row = {
            'matching_team': cand['team_name'],
            'user_type': int(user_type),
            'recommend_league': json_league_name,
            'sbert_score': s_sem, 'n2v_score': s_rel, 'vector_score': s_vec,
            'manual_match_score': manual_match_score
        }
        for i, col in enumerate(pca_cols): row[col] = query_pca[i]
        rows.append(row)

    df_inf = pd.DataFrame(rows)
    df_inf['recommend_league_enc'] = le_league.transform(df_inf['recommend_league'].astype(str))
    
    # 스케일링 및 AI 모델 예측
    score_cols = ['sbert_score', 'n2v_score', 'vector_score']
    scaled = scaler.transform(df_inf[score_cols])
    df_inf['sbert_score_mm'], df_inf['n2v_score_mm'], df_inf['vector_score_mm'] = scaled[:,0], scaled[:,1], scaled[:,2]

    X_input = df_inf[input_features]
    cat_cols = [c for c in ["user_type", "recommend_league_enc"] if c in input_features]
    for c in cat_cols: X_input[c] = X_input[c].astype('category')

    df_inf['predict_score'] = final_model.predict(X_input)

    # 최종 8:2 하이브리드 결합
    df_inf['final_hybrid_score'] = (df_inf['manual_match_score'] * 0.8) + (df_inf['predict_score'] * 0.2)

    return df_inf.sort_values(by='final_hybrid_score', ascending=False).head(5)

# ---------------------------------------------------------
# 5. 실행부
# ---------------------------------------------------------
print("\n" + "="*50)
user_q = input("💬 선호 스타일 입력: ")
ut = input("💬 팬 여부 (1/0): ")
my_team = input("💬 현재 응원팀: ") if ut == '1' else None
target = input("💬 추천받을 리그 (EPL/K리그/KBO/F1): ")

res = recommend_service(user_q, ut, my_team, target)

if isinstance(res, str):
    print(f"\n⚠️ {res}")
else:
    print(f"\n✨ 하이브리드 추천 결과:")
    for i, row in enumerate(res.itertuples(), 1):
        print(f"{i}위: {row.matching_team} [종합: {row.final_hybrid_score:.4f} (시나리오: {row.manual_match_score:.4f} / AI: {row.predict_score:.4f})]")

🔍 SBERT 모델(KR-SBERT) 로딩 중...
🔍 PCA 인코더 모델(ko-sroberta) 로딩 중...
✅ 모델 및 전처리 아티팩트 로드 완료
🚀 팀 네트워크 분석 중...


✨ 하이브리드 추천 결과:
1위: 애스턴 마틴 [종합: 0.1830 (시나리오: 0.3682 / AI: -0.5578)]
2위: 메르세데스 [종합: 0.0377 (시나리오: 0.3865 / AI: -1.3575)]
3위: 페라리 [종합: 0.0324 (시나리오: 0.4584 / AI: -1.6714)]
4위: 레이싱 불스 [종합: 0.0273 (시나리오: 0.5012 / AI: -1.8683)]
5위: 레드불 [종합: 0.0254 (시나리오: 0.5073 / AI: -1.9022)]


In [37]:
# !pip install hf_xet

In [50]:
import os
import json
import pandas as pd
import numpy as np
import networkx as nx
import joblib
import warnings
from node2vec import Node2Vec
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings("ignore")

# ---------------------------------------------------------
# 1. 모델 및 환경 설정 (이원화 로딩)
# ---------------------------------------------------------
DATA_DIR = r'./JSON 모음'

# [모델 1] s_sem(의미 유사도) 전용: 기존 SBERT 유지
print("🔍 s_sem 계산용 SBERT 모델(KR-SBERT) 로딩 중...")
model_sbert = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# [모델 2] PCA(의도 분석) 전용: 최신 E5 모델 적용
print("🔍 PCA 인코더용 E5 모델(multilingual-e5-base) 로딩 중...")
model_pca_encoder = SentenceTransformer('intfloat/multilingual-e5-base')

ALPHA, BETA, GAMMA = 0.4, 0.4, 0.2

LEAGUE_MAP = {
    "K리그": "K league", "EPL": "EPL", "KBO": "KBO", "F1": "F1"
}

try:
    artifacts = joblib.load('sports_chatbot_model2.joblib')
    final_model = artifacts['final_model']
    pca = artifacts['pca']
    scaler = artifacts['scaler']
    le_league = artifacts['le_league']
    input_features = artifacts['input_features']
    print("✅ 모델 및 전처리 아티팩트 로드 완료")
except Exception as e:
    print(f"❌ 로드 실패: {e}")
    exit()

# ---------------------------------------------------------
# 2. 데이터 로딩 및 N2V 구축 (기존 유지)
# ---------------------------------------------------------
def load_teams(path):
    teams = []
    if not os.path.exists(path): return teams
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith('.json'):
                with open(os.path.join(root, filename), 'r', encoding='utf-8') as f:
                    teams.append(json.load(f))
    return teams

def build_n2v_model(teams_data):
    G = nx.Graph()
    for t in teams_data:
        if 'team_name' in t: G.add_node(t['team_name'])
    for i in range(len(teams_data)):
        for j in range(i + 1, len(teams_data)):
            tags1 = set(teams_data[i].get('style_tags', []))
            tags2 = set(teams_data[j].get('style_tags', []))
            common = len(tags1.intersection(tags2))
            if common > 0:
                G.add_edge(teams_data[i]['team_name'], teams_data[j]['team_name'], weight=common)
    n2v = Node2Vec(G, dimensions=64, walk_length=10, num_walks=40, workers=1, quiet=True)
    return n2v.fit(window=10, min_count=1)

print("🚀 팀 데이터를 분석하고 관계망을 구축 중입니다...")
teams_master = load_teams(DATA_DIR)
n2v_model = build_n2v_model(teams_master)

# ---------------------------------------------------------
# 3. 실시간 점수 계산 (SBERT 기반 + 리더님 시나리오 로직)
# ---------------------------------------------------------
def get_scores_strict(user_query, anchor_name, candidate):
    # (1) sbert_score: 기존 SBERT 모델로 의미 유사도 계산
    cand_tags = " ".join(candidate.get('style_tags', []))
    embs = model_sbert.encode([user_query, cand_tags])
    s_sem = cosine_similarity([embs[0]], [embs[1]])[0][0]

    # (2) n2v_score: 관계 유사도
    s_rel = 0.5
    if anchor_name and anchor_name in n2v_model.wv and candidate['team_name'] in n2v_model.wv:
        s_rel = n2v_model.wv.similarity(anchor_name, candidate['team_name'])

    # (3) vector_score: [리더님의 시나리오 로직 반영]
    ts = candidate.get('scores', {})
    metrics = ['strength', 'money', 'star_power', 'attack_style', 'underdog_feel', 'fan_passion', 'tradition']
    t_vec = np.array([ts.get(m, 10) for m in metrics])
    
    target_vec = np.array([10]*7)
    s_multiplier = 1.0

    # 리더님의 시나리오 조건문 (수정 없음)
    if any(k in user_query for k in ["언더독", "기적", "저비용", "머니볼", "효율", "가성비"]):
        target_vec[4] = 50
        if ts.get('money', 10) >= 16: s_multiplier = 0.3
        elif ts.get('money', 10) <= 8: s_multiplier = 1.5
    elif any(k in user_query for k in ["강한", "압도적", "최강", "우승", "부자"]):
        target_vec[0], target_vec[1] = 40, 40
        if ts.get('strength', 10) < 12: s_multiplier = 0.4
    elif any(k in user_query for k in ["미남", "잘생긴", "비주얼", "얼굴", "입덕"]):
        target_vec[2] = 50
        if ts.get('star_power', 10) < 12: s_multiplier = 0.4
    elif any(k in user_query for k in ["전통", "명문", "역사", "연고지", "자부심"]):
        target_vec[6], target_vec[5] = 40, 40
        if ts.get('tradition', 10) < 10: s_multiplier = 0.5
    elif any(k in user_query for k in ["공격", "화끈", "득점", "홈런", "추월", "시원시원"]):
        target_vec[3] = 50
        if ts.get('attack_style', 10) < 12: s_multiplier = 0.5
    elif any(k in user_query for k in ["수비", "단단한", "실리", "역습", "질식"]):
        target_vec[4], target_vec[0] = 30, 30
        if ts.get('attack_style', 10) > 15: s_multiplier = 0.5

    s_vec = cosine_similarity(target_vec.reshape(1, -1), t_vec.reshape(1, -1))[0][0]
    s_vec = s_vec * s_multiplier
    
    return s_sem, s_rel, s_vec

# ---------------------------------------------------------
# 4. 추천 서비스 (E5 접두사 + 하이브리드 로직)
# ---------------------------------------------------------
def recommend_service(query, user_type, support_team, target_league):
    json_league_name = LEAGUE_MAP.get(target_league, target_league)
    candidates = [t for t in teams_master if t.get('league', '').lower() == json_league_name.lower()]
    
    if not candidates: return f"'{target_league}' 리그의 팀 데이터를 찾을 수 없습니다."

    # [수정] E5 모델 특성상 질문 앞에 'query: ' 접두사를 추가하여 벡터화 수행
    e5_query = f"query: {query}"
    query_pca_vec = model_pca_encoder.encode([e5_query])
    query_pca = pca.transform(query_pca_vec)[0]
    
    pca_cols = ["의도_팬덤정체성", "의도_스타성과강함", "의도_명문과기적", "의도_비주얼과매력", "의도_자본과지배력"]

    rows = []
    for cand in candidates:
        s_sem, s_rel, s_vec = get_scores_strict(query, support_team, cand)
        
        # 수동 점수 계산 (80% 반영될 핵심 점수)
        manual_match_score = (s_sem * ALPHA) + (s_rel * BETA) + (s_vec * GAMMA)

        row = {
            'matching_team': cand['team_name'],
            'user_type': int(user_type),
            'recommend_league': json_league_name,
            'sbert_score': s_sem, 'n2v_score': s_rel, 'vector_score': s_vec,
            'manual_match_score': manual_match_score
        }
        for i, col in enumerate(pca_cols): row[col] = query_pca[i]
        rows.append(row)

    df_inf = pd.DataFrame(rows)
    df_inf['recommend_league_enc'] = le_league.transform(df_inf['recommend_league'].astype(str))
    
    # MinMaxScaler 적용 및 AI 모델 예측
    score_cols = ['sbert_score', 'n2v_score', 'vector_score']
    scaled = scaler.transform(df_inf[score_cols])
    df_inf['sbert_score_mm'], df_inf['n2v_score_mm'], df_inf['vector_score_mm'] = scaled[:,0], scaled[:,1], scaled[:,2]

    X_input = df_inf[input_features]
    cat_cols = [c for c in ["user_type", "recommend_league_enc"] if c in input_features]
    for c in cat_cols: X_input[c] = X_input[c].astype('category')

    df_inf['predict_score'] = final_model.predict(X_input)

    # 최종 8:2 하이브리드 결합 (수동 로직 0.8, 학습 모델 0.2)
    df_inf['final_hybrid_score'] = (df_inf['manual_match_score'] * 0.9) + (df_inf['predict_score'] * 0.1)

    return df_inf.sort_values(by='final_hybrid_score', ascending=False).head(5)

# ---------------------------------------------------------
# 5. 실행부
# ---------------------------------------------------------
print("\n" + "="*50)
print("       🏆 스포츠 팀 하이브리드 추천 엔진 (E5 적용)")
print("="*50)

user_q = input("💬 원하시는 팀의 분위기나 특징을 입력해주세요: ")
ut = input("💬 스포츠 팬 여부 (1:팬, 0:입문자): ")
my_team = input("💬 현재 응원 중인 팀 (없으면 엔터): ") if ut == '1' else None
target = input("💬 추천 리그 선택 (EPL/K리그/KBO/F1): ")

res = recommend_service(user_q, ut, my_team, target)

if isinstance(res, str):
    print(f"\n⚠️ {res}")
else:
    print(f"\n✨ '{target}' 리그 하이브리드 분석 결과:")
    for i, row in enumerate(res.itertuples(), 1):
        star = "⭐" if i <= 3 else "  "
        print(f"{star} {i}위: {row.matching_team}")
        print(f"    - 종합 점수: {row.final_hybrid_score:.4f} (시나리오: {row.manual_match_score:.4f} / AI 예측: {row.predict_score:.4f})")

🔍 s_sem 계산용 SBERT 모델(KR-SBERT) 로딩 중...
🔍 PCA 인코더용 E5 모델(multilingual-e5-base) 로딩 중...
✅ 모델 및 전처리 아티팩트 로드 완료
🚀 팀 데이터를 분석하고 관계망을 구축 중입니다...

       🏆 스포츠 팀 하이브리드 추천 엔진 (E5 적용)

✨ 'K LEAGUE' 리그 하이브리드 분석 결과:
⭐ 1위: 전북 현대 모터스
    - 종합 점수: 0.2786 (시나리오: 0.3362 / AI 예측: -0.2401)
⭐ 2위: 수원 삼성 블루윙즈
    - 종합 점수: 0.2578 (시나리오: 0.4821 / AI 예측: -1.7609)
⭐ 3위: 대구FC
    - 종합 점수: 0.2573 (시나리오: 0.5780 / AI 예측: -2.6290)
   4위: 강원FC
    - 종합 점수: 0.2368 (시나리오: 0.5533 / AI 예측: -2.6120)
   5위: 인천 유나이티드
    - 종합 점수: 0.2276 (시나리오: 0.4815 / AI 예측: -2.0577)
